In [39]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from sklearn.metrics import r2_score
import plotly.express as px
import plotly.graph_objects as go

In [25]:
# Dataset with Next Coupon Date, Previous Coupon Date, Invoice Price, Time(Years)
# This was computed in Excel before starting the project in Python
df = pd.read_excel("MP_02.xlsx")

df.head()

,Issuer Name,Issue Date,Maturity,Bid Price,Ask Price,Cpn,Currency,Clean price,Days to Maturity,Accrued Interest,Next Coupon Date,Previous Coupon Date,Invoice Price,Time(Years),Present Value
0,United States Treasury Note/Bond,2/28/2018,2/28/2025,99.980469,100.007812,2.750,USD,99.994141,4,1.345109,2025-02-28,2024-08-28,101.339249,0.010951,NaN
1,United States Treasury Note/Bond,2/28/2023,2/28/2025,99.984375,100.019531,4.625,USD,100.001953,4,2.262228,2025-02-28,2024-08-28,102.264181,0.010951,NaN
2,United States Treasury Note/Bond,3/2/2020,2/28/2025,99.945312,100.011719,1.125,USD,99.978516,4,0.550272,2025-02-28,2024-08-28,100.528787,0.010951,NaN
3,United States Treasury Note/Bond,3/15/2022,3/15/2025,99.859375,99.914062,1.750,USD,99.886719,19,0.783149,2025-03-15,2024-09-15,100.669868,0.052019,NaN
4,United States Treasury Note/Bond,3/31/2020,3/31/2025,99.621094,99.703125,0.500,USD,99.662109,35,0.201923,2025-03-31,2024-09-30,99.864032,0.095825,NaN


In [26]:
df["Maturity"] = pd.to_datetime(df["Maturity"])
df["Issue Date"] = pd.to_datetime(df["Issue Date"])
df["Next Coupon Date"] = pd.to_datetime(df["Next Coupon Date"])
df["Previous Coupon Date"] = pd.to_datetime(df["Previous Coupon Date"])

df.head()

,Issuer Name,Issue Date,Maturity,Bid Price,Ask Price,Cpn,Currency,Clean price,Days to Maturity,Accrued Interest,Next Coupon Date,Previous Coupon Date,Invoice Price,Time(Years),Present Value
0,United States Treasury Note/Bond,2018-02-28,2025-02-28,99.980469,100.007812,2.750,USD,99.994141,4,1.345109,2025-02-28,2024-08-28,101.339249,0.010951,NaN
1,United States Treasury Note/Bond,2023-02-28,2025-02-28,99.984375,100.019531,4.625,USD,100.001953,4,2.262228,2025-02-28,2024-08-28,102.264181,0.010951,NaN
2,United States Treasury Note/Bond,2020-03-02,2025-02-28,99.945312,100.011719,1.125,USD,99.978516,4,0.550272,2025-02-28,2024-08-28,100.528787,0.010951,NaN
3,United States Treasury Note/Bond,2022-03-15,2025-03-15,99.859375,99.914062,1.750,USD,99.886719,19,0.783149,2025-03-15,2024-09-15,100.669868,0.052019,NaN
4,United States Treasury Note/Bond,2020-03-31,2025-03-31,99.621094,99.703125,0.500,USD,99.662109,35,0.201923,2025-03-31,2024-09-30,99.864032,0.095825,NaN


In [27]:
# Function to expand coupon payments to include calculated -> payment dates, unique_bond_id, unique_coupon_id (corresponding to bond)
def expand_bond_payments(row, idx):
    payments = []
    coupon_interval = pd.DateOffset(months = 6)
    payment_date = row["Next Coupon Date"]
    counter = 1
    while payment_date <= row["Maturity"]:
        years_to_payment = (payment_date - pd.Timestamp("2025-02-24")).days / 365.25
        
        payments.append({
            "Issuer Name": row["Issuer Name"],
            "Issue Date" : row["Issue Date"],
            "Maturity" : row["Maturity"],
            "Bid Price" : row["Bid Price"],
            "Ask Price" : row["Ask Price"],
            "Cpn" : row["Cpn"],
            "Currency" : row["Currency"],
            "Clean price" : row["Clean price"],
            "Days to Maturity" : row["Days to Maturity"],
            "Accrued Interest" : row["Accrued Interest"],
            "Next Coupon Date": payment_date,
            "Previous Coupon Date" : row["Previous Coupon Date"],
            "Invoice Price" : row["Invoice Price"],
            "Time(Years)" : row["Invoice Price"],
            "Cpn": row["Cpn"],
            "Time(Years)": years_to_payment,
            "Unique_Bond" : f"{idx + 1}",
            "Unique_Coupon" : f"{counter}",
            "Currency": row["Currency"]
        })
        payment_date += coupon_interval
        counter += 1
    
    return payments

In [28]:
expanded_rows = []
for idx, row in df.iterrows():
    expanded_rows.extend(expand_bond_payments(row, idx))

expanded_df = pd.DataFrame(expanded_rows)

expanded_df.head()

,Issuer Name,Issue Date,Maturity,Bid Price,Ask Price,Cpn,Currency,Clean price,Days to Maturity,Accrued Interest,Next Coupon Date,Previous Coupon Date,Invoice Price,Time(Years),Unique_Bond,Unique_Coupon
0,United States Treasury Note/Bond,2018-02-28,2025-02-28,99.980469,100.007812,2.750,USD,99.994141,4,1.345109,2025-02-28,2024-08-28,101.339249,0.010951,1,1
1,United States Treasury Note/Bond,2023-02-28,2025-02-28,99.984375,100.019531,4.625,USD,100.001953,4,2.262228,2025-02-28,2024-08-28,102.264181,0.010951,2,1
2,United States Treasury Note/Bond,2020-03-02,2025-02-28,99.945312,100.011719,1.125,USD,99.978516,4,0.550272,2025-02-28,2024-08-28,100.528787,0.010951,3,1
3,United States Treasury Note/Bond,2022-03-15,2025-03-15,99.859375,99.914062,1.750,USD,99.886719,19,0.783149,2025-03-15,2024-09-15,100.669868,0.052019,4,1
4,United States Treasury Note/Bond,2020-03-31,2025-03-31,99.621094,99.703125,0.500,USD,99.662109,35,0.201923,2025-03-31,2024-09-30,99.864032,0.095825,5,1


In [29]:
expanded_df.to_excel("expanded_excel.xlsx", index = False)

$$
\left( \sum_{i=1}^{n} c_i \cdot \text{dis} \right) + P \cdot \text{dis} - \text{accrued\_interest}
$$

$$
\text{dis} = e^{-r(t) \cdot t}
$$

$$
r(t) = a + b \cdot t + c \cdot t^{2} + d \cdot e^{f t} + g \cdot \ln{(t - h)}
$$

In [30]:
# continuously compunded zero coupon bond discount rate
def r_t(x, t):
    a, b, c, d, f, g, h = x
    return a + b * t + c * t ** 2 + d * np.e ** (f * t) + g * np.log(t - h)

# discounting factor function
def discount_factor(x, t):
    return np.e ** (-1 * r_t(x, t) * t)

# function to minimize sum of squared errors, fitting a function for calculating parameters for rate function 
# and minimizing squared errors between predicted and actual values
def price_minimize(x):
    net_pred = []
    for i in expanded_df["Unique_Bond"].unique():
        price = 0.0
        df_subset = expanded_df[expanded_df["Unique_Bond"] == i]
        tvals = df_subset["Time(Years)"].values
        cpns = df_subset["Cpn"].values
        price += np.sum((cpns / 2.0) * discount_factor(x, tvals))
        price += 100.0 * discount_factor(x, df_subset["Time(Years)"].values[-1])
        price -= df_subset["Accrued Interest"].values[0]
        net_pred.append(price)
    resid = df["Clean price"].to_numpy() - np.array(net_pred)
    return np.sum(resid ** 2)

In [ ]:
#              a          b        c       d       f        g          h
x0 = np.array([0.2720, 0.0086, -0.0001, 0.0375, 0.0150, -0.1225, -7.9728])

# other different initial values we can test out
# x0 = np.array([-0.18296765, 0.002977243, -7.33536E-05, 0.121977486, -0.086535926, 0.054641797, -6.341670119])
# x0 = np.array([0, 0, 0, 0, 0, 0, 0])
min_time = expanded_df["Time(Years)"].min()
bounds = [(None, None)] * 7
bounds[6] = (None, min_time - 1e-8)

result = minimize(price_minimize, x0, method = 'L-BFGS-B', bounds = bounds, options = {'maxiter' : 10000})
a, b, c, d, f, g, h = result.x

result.x

array([ 2.29697737e-01,  7.51719503e-03, -1.02081862e-04, -6.77053530e-04,
        4.29250132e-02, -8.68951460e-02, -7.97602760e+00])

In [32]:
print("Sum of Squared Errors ->", price_minimize(result.x))

Sum of Squared Errors -> 55.935965996546685


In [33]:
result.message

'CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH'

In [53]:
# a + b * x + c * x ** 2 + d * np.e ** (f * x) + g * np.log(x - h)
pd.DataFrame({"Parameter" : ["a", "b", "c", "d", "f", "g", "h"], "Value" : [a, b, c, d, f, g, h]}).set_index("Parameter").T.to_csv("Parameters.csv")

In [35]:
def build_pred_df(x):
    net_pred = []
    for i in expanded_df["Unique_Bond"].unique():
        price = 0.0
        df_subset = expanded_df[expanded_df["Unique_Bond"] == i]
        tvals = df_subset["Time(Years)"].values
        cpns = df_subset["Cpn"].values
        price += np.sum((cpns / 2.0) * discount_factor(x, tvals))
        price += 100.0 * discount_factor(x, tvals[-1])
        price -= df_subset["Accrued Interest"].values[0]
        net_pred.append(price)
    observed = df["Clean price"].to_numpy()
    pred = np.array(net_pred)
    maturity = df["Maturity"].to_list()
    return pd.DataFrame({"observed" : observed, "pred" : pred, "maturity" : maturity})

results_df = build_pred_df(result.x)
results_df.head()

,observed,pred,maturity
0,99.994141,99.976005,2025-02-28
1,100.001953,99.995887,2025-02-28
2,99.978516,99.958773,2025-02-28
3,99.886719,99.838129,2025-03-15
4,99.662109,99.585474,2025-03-31


In [37]:
x_axis = [0.1 * i for i in range(1, 300)]
y_axis = [r_t(result.x, i) for i in x_axis]

pred_df = pd.DataFrame({"Time" : x_axis, "Continuously Compunded Rate" : y_axis})
px.line(pred_df, x = "Time", y = "Continuously Compunded Rate").update_layout(yaxis = dict(rangemode = "tozero"))

$$
(Bid + Ask) / 2 = \left( \sum_{i=1}^{n} c_i \cdot \text{dis} \right) + P \cdot \text{dis} - \text{accrued\_interest}
$$

In [38]:
print("R^2 score ->", r2_score(results_df["observed"], results_df["pred"]))

R^2 score -> 0.9985829974762582


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df["Time(Years)"], y = [r_t(result.x, i) for i in df["Time(Years)"]], name = "Rate Function", yaxis = "y1", mode = "lines+markers"))
fig.add_trace(go.Scatter(x = df["Time(Years)"], y = results_df["observed"], name = "Observed Clean Prices", yaxis = "y2", mode = "lines+markers"))
fig.update_layout(
    title = f"Rate Function vs Observed Clean Prices [Correlation {np.corrcoef([r_t(result.x, i) for i in df["Time(Years)"]], results_df["observed"])[0][1]}]",
    xaxis = dict(title = "Time (Years)"),
    yaxis = dict(title = "Rate Function"),
    yaxis2 = dict(
        title = "Observed Clean Price",
        overlaying = "y",
        side = "right"
    )
)
fig.show()

In [47]:
px.line(results_df.sort_values(by = "maturity"), x = "maturity", y = ["pred", "observed"]).update_layout(title = f"Actual vs Predicted Values [R^2 {r2_score(results_df["observed"], results_df["pred"])}]")